In [ ]:
!pip install ultralytics
!pip install comet_ml
!pip install pycocotools
from IPython import display
display.clear_output()
import comet_ml
import os
import ultralytics
ultralytics.checks()

In [ ]:
from IPython import display
import comet_ml
import os
import ultralytics
from ultralytics import YOLO
os.environ["COMET_API_KEY"] = "oY1fZVkoMqprClMkFNW33OwqX"
from IPython.display import display, Image

In [ ]:
comet_ml.init("Prova-tracking-YOLO3")
model=YOLO("/kaggle/input/dataset-cv/finaleSaccomannoZumpano.pt")


In [ ]:
risultato=model.train(model="yolov8s-seg.pt", data="/kaggle/input/dataset-cv/Computer_Vision_yolov8/data.yaml",epochs=500,imgsz=736,save=True,close_mosaic=50,project="Prova-tracking-YOLO3",batch=-1,save_period=20,box=10,copy_paste=0.5)
#!yolo task=segment mode=train model=yolov8s-seg.pt data=/kaggle/input/dataset-cv/Computer_Vision_yolov8/data.yaml epochs=50 imgsz=640 save=True close_mosaic=10 overlap_mask= False mask_ratio=1

In [ ]:
import json

images={} # key:nome_file.jpeg 
with open('/kaggle/input/testsetorgin/testset/test.json') as json_file:
    data = json.load(json_file)
    for img in data["images"]:
        images[img['file_name']] = img['id']


In [ ]:
import numpy as np
import pycocotools.mask as maskUtils

In [ ]:
modelloFinale=YOLO("/kaggle/input/dataset-cv/finaleSaccomannoZumpano.pt")

In [ ]:
import os
cartella = '/kaggle/input/testsetorgin/testset'
ret=[]
bad_ann=0
for nome_file in os.listdir(cartella):
    percorso_completo = os.path.join(cartella, nome_file)
    if(nome_file!="test.json"):
        risultato = modelloFinale.predict(percorso_completo)
        n_detections = len(risultato[0].boxes.xywh)
        for i in range(n_detections):
            image_id = images[nome_file]
            category_id=int(risultato[0].boxes[i].cls.tolist()[0]) #mi da un tensore con una lista contente solo un numero ovvero 1.0 l'id
            score = float(risultato[0].boxes[i].conf.tolist()[0])
            bbox=risultato[0].boxes[i].xywh.tolist()[0]
            bbox[0]=bbox[0]-bbox[2]/2
            bbox[1]=bbox[1]-bbox[3]/2
            segm=[risultato[0].masks[i].xy[0].flatten().tolist()]  #lista di lista, ognuna è una coppia xy
            #serve che segm sia una lista di una lista con tutti i punti per essere encodata
            
            if(len(segm[0])==0 or len(segm[0])==4 or len(segm[0])==2): #se len 4 si pensa che è una bbox -.- https://github.com/cocodataset/cocoapi/issues/139
                bad_ann+=1
                continue
            
            rles = maskUtils.frPyObjects(segm, 700, 700)
            rle = maskUtils.merge(rles)
            rle['counts'] = rle['counts'].decode('ascii')             #SERVE PER SCRIVERE IL FILE JSON SE NO NON LO ESPORTA PERCHE JSON.DUMP NON PRENDTE BYTE
            ret.append({
                      'image_id': image_id,
                      'category_id': category_id,
                      'bbox': bbox,
                      'segmentation': rle,
                      'score': score
                    })
            

In [ ]:
import json
with open('file2.json', 'w') as file:
    json.dump(ret, file)

In [ ]:
from pycocotools.cocoeval import COCOeval
from pathlib import Path
from pycocotools.coco import COCO


In [ ]:
def evaluate_object_detection(gt_json_path, pred_json_path, check_cable_class=False):
    # Load ground truth and predictions in COCO format
    coco_gt = COCO(gt_json_path)
    coco_pred = coco_gt.loadRes(pred_json_path)

    # Create COCOeval object
    coco_eval = COCOeval(coco_gt, coco_pred, 'bbox')
    if check_cable_class:
        coco_eval.params.catIds = [0]  # id of the cable class

    # Run evaluation
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    avg_p50 = coco_eval.stats[1]
    return avg_p50

In [ ]:
def evaluate_segmentation(gt_json_path, pred_json_path, check_cable_class=False):
    # Load ground truth
    coco_gt = COCO(gt_json_path)

    # Load predictions
    with open(pred_json_path, 'r') as f:
        predictions = json.load(f)

    # Load results into COCO results structure
    coco_res = coco_gt.loadRes(predictions)

    # Create COCOeval object
    coco_eval = COCOeval(coco_gt, coco_res, 'segm')
    if check_cable_class:
        coco_eval.params.catIds = [0]  # id of the cable class

    # Run evaluation
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    avg_p50 = coco_eval.stats[1]
    return avg_p50

In [ ]:
gt_json_path = '/kaggle/input/testsetorgin/testset/test.json'

In [ ]:
pred_json_path = '/kaggle/working/file2.json'
evaluate_object_detection(gt_json_path, pred_json_path, False)
print('#'*50)
evaluate_segmentation(gt_json_path, pred_json_path, False)

print('#'*50)
print('#'*50)

evaluate_object_detection(gt_json_path, pred_json_path, True)
print('#'*50)
evaluate_segmentation(gt_json_path, pred_json_path, True)